In [70]:
! pip install prophet==1.1.5
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, RationalQuadratic
import joblib
from prophet import Prophet



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: C:\Users\wrksy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached prophet-1.1.5-py3-none-win_amd64.whl.metadata (3.6 kB)
  Using cached cmdstanpy-1.2.4-py3-none-any.whl.metadata (4.1 kB)
  Using cached holidays-0.54-py3-none-any.whl.metadata (23 kB)
  Using cached importlib_resources-6.4.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached stanio-0.5.1-py3-none-any.whl.metadata (1.6 kB)
Using cached prophet-1.1.5-py3-none-win_amd64.whl (13.3 MB)
Using cached cmdstanpy-1.2.4-py3-none-any.whl (94 kB)
Using cached holidays-0.54-py3-none-any.whl (1.0 MB)
Using cached importlib_resources-6.4.0-py3-none-any.whl (38 kB)
Using cached stanio-0.5.1-py3-none-any.whl (8.1 kB)


C:\Users\wrksy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
# Load data
df = pd.read_csv('kc_house_data.csv', encoding='ISO-8859-1')


In [57]:
df.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7.129301e+09,20141013T000000,221900.0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,...,7.0,1180.0,0.0,1955.0,0.0,98178.0,47.5112,-122.257,1340.0,5650.0
1,6.414100e+09,20141209T000000,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,...,7.0,2170.0,400.0,1951.0,1991.0,98125.0,47.7210,-122.319,1690.0,7639.0
2,5.631500e+09,20150225T000000,180000.0,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,...,6.0,770.0,0.0,1933.0,0.0,98028.0,47.7379,-122.233,2720.0,8062.0
3,2.487201e+09,20141209T000000,604000.0,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,...,7.0,1050.0,910.0,1965.0,0.0,98136.0,47.5208,-122.393,1360.0,5000.0
4,1.954401e+09,20150218T000000,510000.0,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,...,8.0,1680.0,0.0,1987.0,0.0,98074.0,47.6168,-122.045,1800.0,7503.0


In [58]:
# Feature engineering
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop(['id', 'date'], axis=1)

# Drop rows where the target variable 'price' is NaN
df = df.dropna(subset=['price'])

In [59]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
0,221900.0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,...,1955.0,0.0,98178.0,47.5112,-122.257,1340.0,5650.0,2014.0,10.0,13.0
1,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,...,1951.0,1991.0,98125.0,47.7210,-122.319,1690.0,7639.0,2014.0,12.0,9.0
2,180000.0,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,...,1933.0,0.0,98028.0,47.7379,-122.233,2720.0,8062.0,2015.0,2.0,25.0
3,604000.0,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5.0,7.0,...,1965.0,0.0,98136.0,47.5208,-122.393,1360.0,5000.0,2014.0,12.0,9.0
4,510000.0,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3.0,8.0,...,1987.0,0.0,98074.0,47.6168,-122.045,1800.0,7503.0,2015.0,2.0,18.0


In [60]:
# Separate features and target variable
X = df.drop('price', axis=1)
y = df['price']

In [61]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,year,month,day
0,221900.0,3.0,1.00,1180.0,5650.0,1.0,0.0,0.0,3.0,7.0,...,1955.0,0.0,98178.0,47.5112,-122.257,1340.0,5650.0,2014.0,10.0,13.0
1,538000.0,3.0,2.25,2570.0,7242.0,2.0,0.0,0.0,3.0,7.0,...,1951.0,1991.0,98125.0,47.7210,-122.319,1690.0,7639.0,2014.0,12.0,9.0
2,180000.0,2.0,1.00,770.0,10000.0,1.0,0.0,0.0,3.0,6.0,...,1933.0,0.0,98028.0,47.7379,-122.233,2720.0,8062.0,2015.0,2.0,25.0
3,604000.0,4.0,3.00,1960.0,5000.0,1.0,0.0,0.0,5.0,7.0,...,1965.0,0.0,98136.0,47.5208,-122.393,1360.0,5000.0,2014.0,12.0,9.0
4,510000.0,3.0,2.00,1680.0,8080.0,1.0,0.0,0.0,3.0,8.0,...,1987.0,0.0,98074.0,47.6168,-122.045,1800.0,7503.0,2015.0,2.0,18.0


In [62]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [63]:
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [64]:
# Create and fit PolynomialFeatures and StandardScaler
poly = PolynomialFeatures(degree=2, include_bias=False)
scaler = StandardScaler()

In [65]:
# Create and fit PolynomialFeatures and StandardScaler
poly = PolynomialFeatures(degree=2, include_bias=False)
scaler = StandardScaler()

In [66]:
X_train_poly = poly.fit_transform(X_train)
X_train_scaled = scaler.fit_transform(X_train_poly)

In [79]:
# Train GaussianProcessRegressor
kernel = C(1.0, (1e-3, 1e3)) * RationalQuadratic(length_scale=1.0, alpha=1.0)
model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, alpha=1e-2)
model.fit(X_train_scaled, y_train)

C:\Users\wrksy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(alpha=0.01,
                         kernel=1**2 * RationalQuadratic(alpha=1, length_scale=1),
                         n_restarts_optimizer=10)

In [71]:
#model = Prophet()
#model.fit(df_forecast)

In [80]:
# Save the model and preprocessing steps
joblib.dump(model, 'best_gpr_model.pkl')
joblib.dump(poly, 'poly_transformer.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [81]:
print(f"Number of features after polynomial transformation: {X_train_poly.shape[1]}")
print(f"Number of features after scaling: {X_train_scaled.shape[1]}")

Number of features after polynomial transformation: 252
Number of features after scaling: 252
